In [ ]:
import concurrent.futures
import subprocess

names_file_path = "a.txt"
input_dir = "Swiping_Up"
output_dir = "z_output"

def process_video(name):
    name = name.strip()
    path = input_dir + "/" + name
    output_folder = output_dir + "/" + name

    overwrite = True
    unique_output_folder = True
    out_filename = "events.h5"
    davis_output = True

    skip_video_output = True
    dvs_exposure = "duration .033"
    output_mode = "dvs128"

    input_frame_rate = 12
    input_slowmotion_factor = 1

    disable_slomo = True
    timestamp_resolution = 0.001
    auto_timestamp_resolution = True

    condition = "Clean"

    thres = 0.2
    sigma = 0.03
    cutoff_hz = 200
    leak_rate_hz = 5.18
    shot_noise_rate_hz = 2.716

    if condition == "Clean":
        thres = 0.2
        sigma = 0.02
        cutoff_hz = 0
        leak_rate_hz = 0
        shot_noise_rate_hz = 0

    elif condition == "Noisy":
        thres = 0.2
        sigma_thres = 0.05
        cutoff_hz = 30
        leak_rate_hz = 0.1
        shot_noise_rate_hz = 5

    v2e_command = ["v2e"]
    v2e_command += ["-i", path]
    v2e_command += ["-o", output_folder]

    if overwrite:
        v2e_command.append("--overwrite")

    v2e_command += ["--unique_output_folder", "{}".format(unique_output_folder).lower()]

    if davis_output:
        v2e_command += ["--davis_output"]
    v2e_command += ["--dvs_h5", out_filename]
    v2e_command += ["--dvs_aedat2", "None"]
    v2e_command += ["--dvs_text", "None"]

    v2e_command += ["--no_preview"]

    if skip_video_output:
        v2e_command += ["--skip_video_output"]
    else:
        v2e_command += ["--dvs_exposure", dvs_exposure]

    v2e_command += ["--input_frame_rate", "{}".format(input_frame_rate)]
    v2e_command += ["--input_slowmotion_factor", "{}".format(input_slowmotion_factor)]

    v2e_command += ["--disable_slomo"]
    v2e_command += ["--auto_timestamp_resolution", "false"]

    v2e_command += ["--pos_thres", "{}".format(thres)]
    v2e_command += ["--neg_thres", "{}".format(thres)]
    v2e_command += ["--sigma_thres", "{}".format(sigma)]
    v2e_command += ["--cutoff_hz", "{}".format(cutoff_hz)]
    v2e_command += ["--leak_rate_hz", "{}".format(leak_rate_hz)]
    v2e_command += ["--shot_noise_rate_hz", "{}".format(shot_noise_rate_hz)]
    v2e_command += [f"--{output_mode}"]

    # Run the command
    final_v2e_command = " ".join(v2e_command)
    print("Start Process: " + name + "\n")
    subprocess.run(final_v2e_command, shell=True)
    print("Processed: " + name + "\n")

# Reading names from the file
with open(names_file_path, "r") as file:
    names = file.readlines()

# Parallelize the processing with ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(process_video, names)
